In [1]:
import io.liquidsoftware.workflow.*
import arrow.core.Either
import kotlinx.coroutines.runBlocking
import java.time.Instant
import java.util.UUID

// Demo state/command
data class DemoState(val id: String) : WorkflowState
data class DemoCommand(val id: String) : UseCaseCommand

// Workflows
class OkWorkflow(override val id: String) : Workflow<DemoState, DemoState>() {
  override suspend fun executeWorkflow(input: DemoState): Either<WorkflowError, WorkflowResult<DemoState>> =
    Either.Right(WorkflowResult(input))
}

class FailWorkflow(override val id: String) : Workflow<DemoState, DemoState>() {
  override suspend fun executeWorkflow(input: DemoState): Either<WorkflowError, WorkflowResult<DemoState>> =
    Either.Left(WorkflowError.ExecutionError("boom"))
}

class SideEffectFail(override val id: String) : Workflow<DemoState, DemoState>() {
  override suspend fun executeWorkflow(input: DemoState): Either<WorkflowError, WorkflowResult<DemoState>> =
    Either.Left(WorkflowError.ExecutionError("side-effect failed"))
}

// Success use case
val successUseCase = useCase<DemoCommand> {
  startWith { cmd -> Either.Right(DemoState(cmd.id)) }
  then(OkWorkflow("ok-step"))
}

// Failure use case with launched side-effect failure
val failUseCase = useCase<DemoCommand> {
  startWith { cmd -> Either.Right(DemoState(cmd.id)) }
  then(OkWorkflow("ok-before"))
  thenLaunch(SideEffectFail("async-side-effect"))
  awaitLaunched() // will record LaunchedFailureEvent
  then(FailWorkflow("fail-step"))
}

// Run both
val successResult = runBlocking { successUseCase.execute(DemoCommand("123")) }
val failResult = runBlocking { failUseCase.execute(DemoCommand("456")) }

// Summaries (pass launchedFailures if you also stash them in context; executions optional)
val successSummary = successResult.toSummary(useCaseName = "SuccessDemo")
val failSummary = failResult.toSummary(useCaseName = "FailDemo")

println("SUCCESS SUMMARY: $successSummary")
println("SUCCESS EVENTS: ${successResult.fold({ emptyList<Event>() }, { it.events })}")

println("\nFAIL SUMMARY: $failSummary")
println("FAIL EVENTS: ${failResult.fold({ emptyList<Event>() }, { it.events })}")

SUCCESS SUMMARY: UseCaseSummary(useCaseName=SuccessDemo, succeeded=true, duration=null, eventsCount=0, executions=[], rootError=null, launchedFailures=[], correlationId=null)
SUCCESS EVENTS: []

FAIL SUMMARY: UseCaseSummary(useCaseName=FailDemo, succeeded=false, duration=PT0.000049S, eventsCount=0, executions=[WorkflowExecution(workflowName=FailWorkflow, workflowId=fail-step, startTime=2026-01-14T13:16:54.229993Z, endTime=2026-01-14T13:16:54.230042Z, succeeded=false)], rootError=RootErrorInfo(type=ExecutionError, message=boom, workflowId=fail-step), launchedFailures=[], correlationId=null)
FAIL EVENTS: []
